In [ ]:
"""
High-fat diet (HFD) phenotype – NHANES feature engineering (condensed)

Input:  nhanes_analytic.csv  (one row per participant, includes HFD_label)
Output: nhanes_HFD_features_engineered.csv, nhanes_HFD_labels.csv
"""

import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

INPUT_CSV = "nhanes_analytic.csv"
TARGET_COL = "HFD_label"

# See Supplementary Table S3 for exact definitions
MEDICATION_CATEGORIES = [
    "ANALGESICS", "ANTACIDS", "ANTICOAGULANTS", "ANTICONVULSANTS",
    "ANTIDEPRESSANTS", "ANTIDIARRHEALS", "ANTIHISTAMINES",
    "ANTIPSORIATICS", "ANTIPSYCHOTICS", "ANTIRHEUMATICS",
    "ANTITUSSIVES", "BRONCHODILATORS", "CEPHALOSPORINS",
    "DECONGESTANTS", "DIURETICS", "EXPECTORANTS", "IMMUNOSTIMULANTS",
    "LAXATIVES", "PENICILLINS", "QUINOLONES", "SULFONAMIDES",
    "TETRACYCLINES", "VASODILATORS", "VASOPRESSORS", "VITAMINS",
]
HEALTH_CONDITIONS = [
    "diabetes", "prediabetes", "asthma", "overweight", "arthritis",
    "hf", "chd", "angina", "ha", "stroke", "anyliver", "cancer",
    "hypertension", "highchol",
]

def handle_outliers(X, lower=0.01, upper=0.01):
    X = X.copy()
    cols = [c for c in X.select_dtypes(include=["float64", "int64"]).columns
            if X[c].nunique(dropna=False) > 10]
    for c in cols:
        try:
            X[c] = winsorize(X[c], limits=[lower, upper])
        except Exception:
            pass
    return X
def impute_missing_mean(X, cols=None):
    X = X.copy()
    
    if cols is None:
        cols = [c for c in X.select_dtypes(include=["float64", "int64"]).columns
                if X[c].nunique(dropna=False) > 10]
    
    for c in cols:
        try:
            X[c] = X[c].fillna(X[c].mean())
        except Exception:
            pass
    
    return X
def add_ethnicity_dummies(X):
    X = X.copy()
    if "RIDRETH1" in X.columns:
        d = pd.get_dummies(X["RIDRETH1"], prefix="ETHNICITY")
        X = pd.concat([X.drop(columns=["RIDRETH1"]), d], axis=1)
    return X

def numeric_interactions(X, max_numeric=8):
    num = X.select_dtypes(include=["float64", "int64"]).columns.tolist()
    if len(num) > max_numeric:
        num = num[:max_numeric]
    data = {f"{a}_x_{b}": X[a] * X[b]
            for i, a in enumerate(num) for b in num[i+1:]}
    return pd.DataFrame(data, index=X.index)

def health_interactions(X):
    data = {}
    health = [c for c in HEALTH_CONDITIONS if c in X.columns]
    num = X.select_dtypes(include=["float64", "int64"]).columns.tolist()

    for i, a in enumerate(health):
        for b in health[i+1:]:
            data[f"{a}_x_{b}"] = X[a] * X[b]

    key_num = [c for c in ["BMXBMI", "avg_sys", "avg_di", "LBXSGL", "LBDHDD"]
               if c in num]
    for h in health:
        for n in key_num:
            data[f"{h}_x_{n}"] = X[h] * X[n]

    return pd.DataFrame(data, index=X.index)

def medication_interactions(X):
    data = {}
    meds = [c for c in MEDICATION_CATEGORIES if c in X.columns]
    health = [c for c in HEALTH_CONDITIONS if c in X.columns]
    demo = [c for c in ["RIDAGEYR", "BMXBMI", "RIAGENDR"] if c in X.columns]

    for m in meds:
        for h in health:
            data[f"{m}_x_{h}"] = X[m] * X[h]
        for d in demo:
            data[f"{m}_x_{d}"] = X[m] * X[d]

    return pd.DataFrame(data, index=X.index)

def medication_burden(X):
    meds = [c for c in MEDICATION_CATEGORIES if c in X.columns]
    if not meds:
        return pd.DataFrame(index=X.index)
    total = X[meds].sum(axis=1)
    return pd.DataFrame({
        "total_medications": total,
        "polypharmacy_5plus": (total >= 5).astype(int),
        "polypharmacy_10plus": (total >= 10).astype(int),
    }, index=X.index)

def health_clusters(X):
    health = [c for c in HEALTH_CONDITIONS if c in X.columns]
    if not health:
        return pd.DataFrame(index=X.index)
    data = {}
    cardio = [c for c in ["hf", "chd", "angina", "ha", "stroke", "hypertension"]
              if c in health]
    metabolic = [c for c in ["diabetes", "prediabetes", "overweight", "highchol"]
                 if c in health]
    if cardio:
        data["cardio_burden"] = X[cardio].sum(axis=1)
    if metabolic:
        data["metabolic_burden"] = X[metabolic].sum(axis=1)
    data["total_health_burden"] = X[health].sum(axis=1)
    return pd.DataFrame(data, index=X.index)

def health_biomarker_interactions(X):
    data = {}
    if "diabetes" in X.columns and "LBXSGL" in X.columns:
        data["diabetes_glucose"] = X["diabetes"] * X["LBXSGL"]
    if "prediabetes" in X.columns and "LBXSGL" in X.columns:
        data["prediabetes_glucose"] = X["prediabetes"] * X["LBXSGL"]

    cardio_cond = [c for c in ["hf", "chd", "angina", "hypertension"] if c in X.columns]
    cardio_markers = [c for c in ["avg_sys", "avg_di", "LBXSCH", "LBXSLDSI", "LBDHDD"]
                      if c in X.columns]
    for h in cardio_cond:
        for m in cardio_markers:
            data[f"{h}_{m}"] = X[h] * X[m]

    if "anyliver" in X.columns:
        for m in [c for c in ["LBXSAPSI", "LBXSATSI"] if c in X.columns]:
            data[f"anyliver_{m}"] = X["anyliver"] * X[m]

    return pd.DataFrame(data, index=X.index)

def ratio_features(X):
    num = X.select_dtypes(include=["float64", "int64"]).columns.tolist()
    def ok(a, b): return a in num and b in num
    data = {}
    if ok("LBXSGTSI", "LBXSAPSI"):
        data["LBXSGTSI_div_LBXSAPSI"] = X["LBXSGTSI"] / (X["LBXSAPSI"] + 1e-8)
    if ok("LBXSUA", "LBXSCR"):
        data["LBXSUA_div_LBXSCR"] = X["LBXSUA"] / (X["LBXSCR"] + 1e-8)
    if ok("avg_sys", "avg_di"):
        data["avg_sys_div_avg_di"] = X["avg_sys"] / (X["avg_di"] + 1e-8)
    if ok("LBXSLDSI", "LBDHDD"):
        data["LBXSLDSI_div_LBDHDD"] = X["LBXSLDSI"] / (X["LBDHDD"] + 1e-8)
    if ok("BMXBMI", "RIDAGEYR"):
        data["BMXBMI_div_RIDAGEYR"] = X["BMXBMI"] / (X["RIDAGEYR"] + 1e-8)
    return pd.DataFrame(data, index=X.index)

def statistical_features(X):
    data = {}
    blood = [c for c in ["LBXSGTSI", "LBXSIR", "LBXSKSI", "LBXSLDSI"] if c in X.columns]
    if len(blood) >= 3:
        data["blood_work_mean"] = X[blood].mean(axis=1)
        data["blood_work_std"] = X[blood].std(axis=1)
        data["blood_work_median"] = X[blood].median(axis=1)
    if "avg_sys" in X.columns and "avg_di" in X.columns:
        bp = X[["avg_sys", "avg_di"]]
        data["blood_pressure_mean"] = bp.mean(axis=1)
        data["blood_pressure_ratio"] = bp["avg_sys"] / (bp["avg_di"] + 1e-8)
    return pd.DataFrame(data, index=X.index)

def engineer_features_before_shap(X_raw):
    X = handle_outliers(X_raw)
    X = impute_missing_mean(X)
    X = add_ethnicity_dummies(X)
    blocks = [
        numeric_interactions(X),
        health_interactions(X),
        medication_interactions(X),
        medication_burden(X),
        health_clusters(X),
        health_biomarker_interactions(X),
        ratio_features(X),
        statistical_features(X),
    ]
    X_all = pd.concat([X] + blocks, axis=1)
    return X_all.loc[:, ~X_all.columns.duplicated()]

if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    if TARGET_COL not in df.columns:
        raise ValueError(f"Target column '{TARGET_COL}' not found.")
    y = df[TARGET_COL].copy()
    X_raw = df.drop(columns=[TARGET_COL])
    X_features = engineer_features_before_shap(X_raw)
    X_features.to_csv("nhanes_HFD_features_engineered.csv", index=False)
    y.to_csv("nhanes_HFD_labels.csv", index=False)
    print("Saved engineered features and labels.")


Saved feature list to: /blue/guoj1/yayunyeh/highfatdiet/Preprocessed/feature_list_cutoff10_for_supplement.csv
Number of features: 141
